### PROCESO ETL

In [241]:
import pandas as pd
import numpy as py
import json

In [242]:
# Cargar los datasets
movies = pd.read_csv('movies_dataset.csv')
credits = pd.read_csv('credits.csv')

C:\Users\54370\AppData\Local\Temp\ipykernel_10796\1638304305.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_dataset.csv')


In [243]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [244]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


### Función para convertir una columna a solo datos numericos tipo Int64 o float

In [245]:
def convertir_tipo(df, column, type):
    """
    Filtra los valores que contienen solo números en la columna especificada,
    elimina las filas no numéricas y convierte la columna a tipo int.

    Args:
    df (pd.DataFrame): El DataFrame a procesar.
    column (str): El nombre de la columna a convertir.

    Returns:
    pd.DataFrame: El DataFrame procesado con la columna solo numérica.
    """
    df = df.copy()
    
    # Filtrar los valores que contienen solo números
    df['numeric'] = df[column].str.extract('(\d+)', expand=False)
    
    # Eliminar las filas que no son numéricas
    df = df.dropna(subset=['numeric'])
    
    # Convertir la columna a tipo int
    df['numeric'] = df['numeric'].astype(type)
    
    # Eliminar la columna original y renombrar la nueva columna
    df = df.drop(columns=[column])
    df = df.rename(columns={'numeric': column})
    
    return df

La columna movies['id'] es de tipo objeto, lo pasamos a Int64.  

La columna contiene valores numericos y no numericos que son incorrectos para un id, para lo cual procedemos a eliminar todos los valores que no sean numericos y luego verificamos si exiten duplicados.

In [246]:
movies['id'].dtype

dtype('O')

In [247]:
movies = convertir_tipo(movies, 'id', 'Int64')


In [248]:
movies['id'].dtype

Int64Dtype()

Mover columna Id al inicio del dataframe

In [249]:
# Especificar la columna que quieres mover al inicio
col_to_move = 'id'

# Obtener una lista de todas las columnas
cols = list(movies.columns)

# Remover la columna que quieres mover al inicio de la lista
cols.remove(col_to_move)

# Insertar la columna al inicio de la lista
cols.insert(0, col_to_move)

# Reorganizar el DataFrame con el nuevo orden de columnas
movies = movies[cols]

movies.head(2)

,id,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,862,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,8844,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


### Chequeo de nulos

In [250]:
def chequear_nulos(df, column):
    """
    Verifica si hay datos nulos en la columna especificada de un DataFrame y
    imprime un mensaje indicando si existen o no datos nulos.

    Args:
    df (pd.DataFrame): El DataFrame a verificar.
    column (str): El nombre de la columna a chequear.
    """
    if df[column].isnull().any():
        print(f"Hay datos nulos en la columna '{column}'.")
    else:
        print(f"No hay datos nulos en la columna '{column}'.")

In [251]:
# Chequeamos datos nulos en id
chequear_nulos(movies, 'id')

No hay datos nulos en la columna 'id'.


### Chequeo de duplicados

In [252]:
def chequear_duplicados(df, column):
    """
    Verifica si hay datos duplicados en la columna especificada de un DataFrame y
    imprime un mensaje indicando si existen o no datos duplicados.

    Args:
    df (pd.DataFrame): El DataFrame a verificar.
    column (str): El nombre de la columna a chequear.
    """
    # Verificar si hay datos duplicados
    hay_datos_duplicados = df[column].duplicated().any()
    
    # Contar el número de duplicados
    num_duplicados = df[column].duplicated().sum()
    
    if hay_datos_duplicados:
        print(f"Hay {num_duplicados} datos duplicados en la columna '{column}'.")
    else:
        print(f"No hay datos duplicados en la columna '{column}'.")

In [253]:
# Chequear si hay datos duplicados en la columna 'id'
chequear_duplicados(movies, 'id')

Hay 33 datos duplicados en la columna 'id'.


### Eliminar duplicados
Se eliminan los valores duplicados y quedan las filas que contengan más campos sin nulos.

In [254]:
# Número de filas antes de eliminar duplicados
num_filas_antes = len(movies)

# Eliminar duplicados quedándote con la fila con más datos no nulos
movies = movies.loc[movies.groupby('id').apply(lambda x: x.notnull().sum(axis=1).idxmax())]

# Número de filas después de eliminar duplicados
num_filas_despues = len(movies)

# Calcular el número de duplicados eliminados
num_duplicados_eliminados = num_filas_antes - num_filas_despues

# Imprimir el DataFrame resultante y el número de duplicados eliminados
print("Data sin duplicados, conservando las filas con más datos no nulos")
print(f"Número de duplicados eliminados: {num_duplicados_eliminados}")

Data sin duplicados, conservando las filas con más datos no nulos
Número de duplicados eliminados: 33


C:\Users\54370\AppData\Local\Temp\ipykernel_10796\2558724696.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  movies = movies.loc[movies.groupby('id').apply(lambda x: x.notnull().sum(axis=1).idxmax())]


In [255]:
# Chequeamos datos nulos en id
chequear_duplicados(movies, 'id')

No hay datos duplicados en la columna 'id'.


### Transformaciones

In [256]:

# Rellenar los valores nulos en las columnas 'revenue' y 'budget' con 0
movies['revenue'] = movies['revenue'].fillna(0)
movies['budget'] = movies['budget'].fillna(0)


In [257]:
# Convertir las columnas a tipo float64
movies = convertir_tipo(movies, 'budget', 'float64')

In [258]:
movies['revenue'].dtype

dtype('float64')

In [259]:
movies['budget'].dtype

dtype('float64')

In [260]:
# Chequeamos datos nulos en id
print(chequear_nulos(movies, 'budget'))
print(chequear_nulos(movies, 'revenue'))


No hay datos nulos en la columna 'budget'.
None
No hay datos nulos en la columna 'revenue'.
None


Transformar fechas: 'release_date' object a datetime

In [261]:
def procesar_release_date(df):
    """
    Procesa la columna 'release_date' para:
    1. Eliminar valores nulos.
    2. Asegurarse de que las fechas estén en el formato AAAA-mm-dd.
    3. Crear una nueva columna 'release_year' que extrae el año de la fecha de estreno.

    Args:
    df (pd.DataFrame): El DataFrame a procesar.

    Returns:
    pd.DataFrame: El DataFrame procesado.
    """
    df = df.copy()
    
    # Eliminar valores nulos en 'release_date'
    df = df.dropna(subset=['release_date'])
    
    # Convertir 'release_date' a datetime, forzando el formato AAAA-mm-dd
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', format='%Y-%m-%d')
    
    # Eliminar filas con fechas no válidas
    df = df.dropna(subset=['release_date'])
    
    # Crear la columna 'release_year' extrayendo el año de 'release_date'
    df['release_year'] = df['release_date'].dt.year
    
    return df

In [262]:
# Procesar la columna 'release_date'
movies = procesar_release_date(movies)

print("DataFrame procesado:")
movies

DataFrame procesado:


,id,adult,belongs_to_collection,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,budget,release_year
4342,2,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,...,69.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}, {'iso_6...",Released,NaN,Ariel,False,7.1,44.0,0.0,1988
12947,3,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",2.29211,...,76.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Shadows in Paradise,False,7.1,35.0,0.0,1986
17,5,False,NaN,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,...,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539.0,4000000.0,1995
474,6,False,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",5.538671,...,110.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.4,79.0,0.0,1993
256,11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,...,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.1,6778.0,11000000.0,1977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45078,465044,False,NaN,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",NaN,tt5943940,en,Abduction,A horror comedy spoofing conspiracy theory mov...,0.281008,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Horrifically Funny,Abduction,False,0.0,0.0,0.0,2017
45273,467731,False,NaN,"[{'id': 18, 'name': 'Drama'}]",NaN,tt0507700,en,Tragedy in a Temporary Town,Fifteen-year-old girl Dotty Fisher is assaulte...,0.001189,...,60.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Tragedy in a Temporary Town,False,0.0,0.0,0.0,1956
21891,468343,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,tt0133202,fi,Silja - nuorena nukkunut,"In the 1910s, beautiful young Silja loses both...",0.001202,...,87.0,[],Released,NaN,Silja - nuorena nukkunut,False,0.0,0.0,0.0,1956
45398,468707,False,NaN,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",http://lmtr.fi/,tt5742932,fi,Lauri Mäntyvaaran tuuheet ripset,NaN,0.347806,...,90.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,NaN,Thick Lashes of Lauri Mäntyvaara,False,8.0,1.0,1254040.0,2017


In [263]:
movies['release_date']

4342    1988-10-21
12947   1986-10-16
17      1995-12-09
474     1993-10-15
256     1977-05-25
           ...    
45078   2017-06-28
45273   1956-02-19
21891   1956-01-01
45398   2017-07-28
20189   1984-08-02
Name: release_date, Length: 45346, dtype: datetime64[ns]

In [264]:
movies['release_year']

4342     1988
12947    1986
17       1995
474      1993
256      1977
         ... 
45078    2017
45273    1956
21891    1956
45398    2017
20189    1984
Name: release_year, Length: 45346, dtype: int32

### Eliminar columnas innecesarias

In [265]:
# Eliminar las columnas no utilizadas
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
movies = movies.drop(columns=columns_to_drop)

In [266]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 4342 to 20189
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45346 non-null  Int64         
 1   belongs_to_collection  4485 non-null   object        
 2   genres                 45346 non-null  object        
 3   original_language      45335 non-null  object        
 4   overview               44405 non-null  object        
 5   popularity             45346 non-null  object        
 6   production_companies   45346 non-null  object        
 7   production_countries   45346 non-null  object        
 8   release_date           45346 non-null  datetime64[ns]
 9   revenue                45346 non-null  float64       
 10  runtime                45100 non-null  float64       
 11  spoken_languages       45346 non-null  object        
 12  status                 45266 non-null  object        
 13  tag

In [267]:
# Crear la columna 'return' manejando los casos donde 'budget' es 0 o NaN
movies['return'] = movies.apply(
    lambda row: row['revenue'] / row['budget'] if pd.notnull(row['revenue']) and pd.notnull(row['budget']) and row['budget'] != 0 else 0,
    axis=1
)

In [268]:
movies.shape

(45346, 20)

In [269]:
movies.columns

Index(['id', 'belongs_to_collection', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'budget', 'release_year', 'return'],
      dtype='object')

### Convertir columnas numericas al formato correcto, chequear nulos

In [270]:
movies.popularity

4342      3.860491
12947      2.29211
17        9.026586
474       5.538671
256      42.149697
           ...    
45078     0.281008
45273     0.001189
21891     0.001202
45398     0.347806
20189     0.001097
Name: popularity, Length: 45346, dtype: object

In [271]:
# col popularity

movies.popularity.dtype

dtype('O')

In [272]:
# Convertir la columna 'popularity' de object a float
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce')

In [273]:
movies.popularity.dtype

dtype('float64')

In [274]:
chequear_nulos(movies, 'popularity')

No hay datos nulos en la columna 'popularity'.


In [275]:
# cols runtime, vote_average, vote_count

print('La columna runtime es de tipo: ',movies.runtime.dtype)
print('La columna vote_average es de tipo: ', movies.vote_average.dtype)
print('La columna vote_count es de tipo: ', movies.vote_count.dtype)



La columna runtime es de tipo:  float64
La columna vote_average es de tipo:  float64
La columna vote_count es de tipo:  float64


In [276]:
chequear_nulos(movies, 'runtime')
chequear_nulos(movies, 'vote_average')
chequear_nulos(movies, 'vote_count')

Hay datos nulos en la columna 'runtime'.
No hay datos nulos en la columna 'vote_average'.
No hay datos nulos en la columna 'vote_count'.


In [277]:
# Verificamos cuantos datos nulos hay en runtime para determinar que criterio optar.
nulos_runtime = movies['runtime'].isna().sum()

print(f"Número de datos nulos en la columna 'runtime': {nulos_runtime}")

Número de datos nulos en la columna 'runtime': 246


Dado que solo una pequeña fracción de los datos tiene valores nulos y el dataset es bastante grande, imputar con la mediana sería un enfoque equilibrado, ya que la mediana es robusta frente a valores atípicos y mantiene la mayoría de los datos intactos sin introducir un sesgo significativo.

In [278]:
median_runtime = movies['runtime'].median()
movies['runtime'].fillna(median_runtime, inplace=True)

C:\Users\54370\AppData\Local\Temp\ipykernel_10796\2548906701.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['runtime'].fillna(median_runtime, inplace=True)


In [279]:
chequear_nulos(movies, 'runtime')


No hay datos nulos en la columna 'runtime'.


In [280]:
movies.shape

(45346, 20)

### Valores NaN 

In [281]:
movies.columns

Index(['id', 'belongs_to_collection', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'budget', 'release_year', 'return'],
      dtype='object')

In [282]:
def contar_valores_nulos(dataframe, columnas):
    for columna in columnas:
        print(f"La columna {columna} tiene: {dataframe[columna].isna().sum()} Valores NaN")

In [283]:
# Lista de columnas a verificar
columnas = [
    'belongs_to_collection',
    'genres',
    'overview',
    'production_companies',
    'production_countries',
    'spoken_languages',
    'status',
    'tagline',
    'title'
]

# Llamada a la función con tu DataFrame y la lista de columnas
contar_valores_nulos(movies, columnas)

La columna belongs_to_collection tiene: 40861 Valores NaN
La columna genres tiene: 0 Valores NaN
La columna overview tiene: 941 Valores NaN
La columna production_companies tiene: 0 Valores NaN
La columna production_countries tiene: 0 Valores NaN
La columna spoken_languages tiene: 0 Valores NaN
La columna status tiene: 80 Valores NaN
La columna tagline tiene: 24959 Valores NaN
La columna title tiene: 0 Valores NaN


**1. belongs_to_collection (40861 Valores NaN)**  

**Contexto:** Esta columna indica si una película pertenece a una colección.  
**Proporción de NaN:** Muy alta (~91%).  
**Recomendación:** Si la columna no es crucial para el análisis o modelo, considera eliminarla. Si crees que puede ser útil, podrías crear una columna binaria que indique si la película pertenece a una colección o no (1 si pertenece, 0 si no pertenece)

In [284]:
# En este caso optamos por crear una columna binaria
movies['belongs_to_collection'] = movies['belongs_to_collection'].notna().astype(int)

In [285]:
movies['belongs_to_collection'].unique()

array([0, 1])

**2. overview (941 Valores NaN)**  
**Contexto:** Esta columna contiene la descripción de la película.  
**Proporción de NaN:** Moderada (~2%).  
**Recomendación:** Dependiendo del análisis, podrías:  
Eliminar filas: Si la columna es crucial y no quieres imputar.  
Imputar con un valor: Podrías imputar con una cadena vacía o un texto como "No overview available".

In [286]:
# Optamos por imputar 'No overview available'
movies['overview'].fillna('No overview available', inplace=True)


C:\Users\54370\AppData\Local\Temp\ipykernel_10796\1573553357.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['overview'].fillna('No overview available', inplace=True)


**3. status (80 Valores NaN)**  
**Contexto:** Esta columna indica el estado de la película (e.g., 'Released', 'Post Production').  
**Proporción de NaN:** Baja (~0.18%).  
**Recomendación:** Podrías imputar con el valor más frecuente (Moda), que probablemente sea 'Released'.

In [287]:
# Imputamos por la Moda
movies['status'].fillna(movies['status'].mode()[0], inplace=True)


C:\Users\54370\AppData\Local\Temp\ipykernel_10796\2253703077.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['status'].fillna(movies['status'].mode()[0], inplace=True)


In [288]:
movies.status

4342     Released
12947    Released
17       Released
474      Released
256      Released
           ...   
45078    Released
45273    Released
21891    Released
45398    Released
20189    Released
Name: status, Length: 45346, dtype: object

**4. tagline (24959 Valores NaN)**  
**Contexto:** Esta columna contiene el eslogan de la película.  
**Proporción de NaN:** Alta (~55%).  
**Recomendación:** Similar a overview, podrías:  
Eliminar filas: Si la columna es crucial y no quieres imputar.  
Imputar con un valor: Podrías imputar con una cadena vacía o un texto como "No tagline".

In [289]:
# Imputamos una cadena vacía
movies['tagline'].fillna(' ', inplace=True)


C:\Users\54370\AppData\Local\Temp\ipykernel_10796\14177463.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['tagline'].fillna(' ', inplace=True)


In [290]:
# Comprobamos de nuevo las columnas si hay NaN

# Llamada a la función con el DataFrame y la lista de columnas
contar_valores_nulos(movies, columnas)

La columna belongs_to_collection tiene: 0 Valores NaN
La columna genres tiene: 0 Valores NaN
La columna overview tiene: 0 Valores NaN
La columna production_companies tiene: 0 Valores NaN
La columna production_countries tiene: 0 Valores NaN
La columna spoken_languages tiene: 0 Valores NaN
La columna status tiene: 0 Valores NaN
La columna tagline tiene: 0 Valores NaN
La columna title tiene: 0 Valores NaN


### Desanidar columnas

In [291]:
movies.iloc[0]['genres']

"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}]"

In [292]:
import ast

# Función para desanidar y concatenar valores
def extract_and_concatenate(column, key):
    def extract_values(value):
        if pd.isna(value):
            return ''
        list_of_dicts = ast.literal_eval(value)
        return ', '.join([d[key] for d in list_of_dicts])
    return column.apply(extract_values)

In [293]:
# Aplica la función a las columnas específicas
movies['genres'] = extract_and_concatenate(movies['genres'], 'name')
movies['production_companies'] = extract_and_concatenate(movies['production_companies'], 'name')
movies['production_countries'] = extract_and_concatenate(movies['production_countries'], 'name')

In [294]:
movies['spoken_languages'] = extract_and_concatenate(movies['spoken_languages'], 'name')

In [295]:
movies['genres'].head()

4342                           Drama, Crime
12947                         Drama, Comedy
17                            Crime, Comedy
474                 Action, Thriller, Crime
256      Adventure, Action, Science Fiction
Name: genres, dtype: object

In [296]:
movies['production_companies'].head()

4342     Villealfa Filmproduction Oy, Finnish Film Foun...
12947                          Villealfa Filmproduction Oy
17                             Miramax Films, A Band Apart
474      Universal Pictures, Largo Entertainment, JVC E...
256      Lucasfilm, Twentieth Century Fox Film Corporation
Name: production_companies, dtype: object

In [297]:
movies['production_countries'].head()

4342                             Finland
12947                            Finland
17              United States of America
474      Japan, United States of America
256             United States of America
Name: production_countries, dtype: object

In [298]:
movies['spoken_languages'].head()

4342              suomi, Deutsch
12947    English, suomi, svenska
17                       English
474                      English
256                      English
Name: spoken_languages, dtype: object

In [299]:
movies.head()

,id,belongs_to_collection,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,budget,release_year,return
4342,2,0,"Drama, Crime",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"Villealfa Filmproduction Oy, Finnish Film Foun...",Finland,1988-10-21,0.0,69.0,"suomi, Deutsch",Released,,Ariel,7.1,44.0,0.0,1988,0.000000
12947,3,0,"Drama, Comedy",fi,"An episode in the life of Nikander, a garbage ...",2.292110,Villealfa Filmproduction Oy,Finland,1986-10-16,0.0,76.0,"English, suomi, svenska",Released,,Shadows in Paradise,7.1,35.0,0.0,1986,0.000000
17,5,0,"Crime, Comedy",en,It's Ted the Bellhop's first night on the job....,9.026586,"Miramax Films, A Band Apart",United States of America,1995-12-09,4300000.0,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,539.0,4000000.0,1995,1.075000
474,6,0,"Action, Thriller, Crime",en,"While racing to a boxing match, Frank, Mike, J...",5.538671,"Universal Pictures, Largo Entertainment, JVC E...","Japan, United States of America",1993-10-15,12136938.0,110.0,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.4,79.0,0.0,1993,0.000000
256,11,1,"Adventure, Action, Science Fiction",en,Princess Leia is captured and held hostage by ...,42.149697,"Lucasfilm, Twentieth Century Fox Film Corporation",United States of America,1977-05-25,775398007.0,121.0,English,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,6778.0,11000000.0,1977,70.490728


In [300]:
movies.columns

Index(['id', 'belongs_to_collection', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'budget', 'release_year', 'return'],
      dtype='object')

Reorganizar columnas

In [301]:
movies = movies[['id', 'title', 'tagline', 'overview', 'runtime',
                        'original_language', 'genres', 'belongs_to_collection',
                        'status', 'release_date', 'release_year',
                        'popularity', 'vote_count', 'vote_average',
                        'production_companies', 'production_countries', 'spoken_languages',
                        'revenue', 'budget', 'return']]

In [302]:
# dataframe movies limpio
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 4342 to 20189
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45346 non-null  Int64         
 1   title                  45346 non-null  object        
 2   tagline                45346 non-null  object        
 3   overview               45346 non-null  object        
 4   runtime                45346 non-null  float64       
 5   original_language      45335 non-null  object        
 6   genres                 45346 non-null  object        
 7   belongs_to_collection  45346 non-null  int64         
 8   status                 45346 non-null  object        
 9   release_date           45346 non-null  datetime64[ns]
 10  release_year           45346 non-null  int32         
 11  popularity             45346 non-null  float64       
 12  vote_count             45346 non-null  float64       
 13  vot

### Dataframe credits

In [303]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [304]:
credits.iloc[0]['cast']

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [305]:
credits.iloc[0]['crew']

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [306]:
credits['crew'].head(3)

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
Name: crew, dtype: object

In [307]:
# merging the 'movies' and 'credits' dataframe based on 'id' column
movies = movies.merge(credits,how='left',on='id')
movies = movies.dropna(subset=['cast','crew'] , axis=0)

In [308]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45389 entries, 0 to 45389
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45389 non-null  Int64         
 1   title                  45389 non-null  object        
 2   tagline                45389 non-null  object        
 3   overview               45389 non-null  object        
 4   runtime                45389 non-null  float64       
 5   original_language      45378 non-null  object        
 6   genres                 45389 non-null  object        
 7   belongs_to_collection  45389 non-null  int64         
 8   status                 45389 non-null  object        
 9   release_date           45389 non-null  datetime64[ns]
 10  release_year           45389 non-null  int32         
 11  popularity             45389 non-null  float64       
 12  vote_count             45389 non-null  float64       
 13  vote_a

In [309]:
# function to get director names from 'crew' column
def get_director_names(row):
    new_list = ast.literal_eval(row)    
    if new_list == [] or isinstance(new_list,float):
        return np.nan
    else:
        inner = []
        for dictionary in new_list:
            job = dictionary.get('job')
            department = dictionary.get('department')
            if((job == 'Director') & (department == 'Directing')):
                inner.append(dictionary['name'])
        return inner

In [310]:
def json_to_list(row,want='name'):
    new_list = ast.literal_eval(row)    
    if new_list == [] or isinstance(new_list,float):
        return (np.nan)
    else:
        inner = []
        for j in new_list:
            inner.append(j[want])
        return (inner)

In [311]:
movies['director'] = movies['crew'].apply(get_director_names)
movies['casting'] = movies['cast'].apply(json_to_list)

In [312]:
movies['director'].head()

0                                     [Aki Kaurismäki]
1                                     [Aki Kaurismäki]
2    [Allison Anders, Alexandre Rockwell, Robert Ro...
3                                    [Stephen Hopkins]
4                                       [George Lucas]
Name: director, dtype: object

In [313]:
movies['casting'].head()

0    [Turo Pajala, Susanna Haavisto, Matti Pellonpä...
1    [Matti Pellonpää, Kati Outinen, Sakari Kuosman...
2    [Tim Roth, Antonio Banderas, Jennifer Beals, M...
3    [Emilio Estevez, Cuba Gooding Jr., Denis Leary...
4    [Mark Hamill, Harrison Ford, Carrie Fisher, Pe...
Name: casting, dtype: object

In [314]:
movies.columns.tolist()

['id',
 'title',
 'tagline',
 'overview',
 'runtime',
 'original_language',
 'genres',
 'belongs_to_collection',
 'status',
 'release_date',
 'release_year',
 'popularity',
 'vote_count',
 'vote_average',
 'production_companies',
 'production_countries',
 'spoken_languages',
 'revenue',
 'budget',
 'return',
 'cast',
 'crew',
 'director',
 'casting']

In [315]:
# Eliminar las columnas 'cast' y 'crew'
movies = movies.drop(columns=['cast', 'crew'])

### Reduccir Dataframe segun popularidad, promedio de votos y cantidad de votos
Debido a las limitaciones del servidor y modo de optimizar el proceso reducimos la cantidad de datos de 45000 a 4500 aproximadamente (nos quedamos con el 10% de las mejores películas). Ya que el objetivo es recomendar peliculas, elegimos de la data original las películas mejores valoradas.

In [316]:
# Obtener los valores máximos y mínimos de las columnas especificadas
max_popularity = movies['popularity'].max()
min_popularity = movies['popularity'].min()

max_vote_count = movies['vote_count'].max()
min_vote_count = movies['vote_count'].min()

max_vote_average = movies['vote_average'].max()
min_vote_average = movies['vote_average'].min()

print(f'Max Popularity: {max_popularity}, Min Popularity: {min_popularity}')
print(f'Max Vote Count: {max_vote_count}, Min Vote Count: {min_vote_count}')
print(f'Max Vote Average: {max_vote_average}, Min Vote Average: {min_vote_average}')

# Ordenar el DataFrame por las columnas 'popularity', 'vote_count', 'vote_average'
movies_sorted = movies.sort_values(by=['popularity', 'vote_count', 'vote_average'], ascending=False)

# Seleccionar el 10% superior del DataFrame ordenado
top_10_percent = int(len(movies) * 0.10)
movies_top_10_percent = movies_sorted.head(top_10_percent)

print(f'Tamaño del DataFrame reducido: {len(movies_top_10_percent)}')


Max Popularity: 547.488298, Min Popularity: 0.0
Max Vote Count: 14075.0, Min Vote Count: 0.0
Max Vote Average: 10.0, Min Vote Average: 0.0
Tamaño del DataFrame reducido: 4538


In [317]:
movies_top_10_percent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4538 entries, 36871 to 2743
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     4538 non-null   Int64         
 1   title                  4538 non-null   object        
 2   tagline                4538 non-null   object        
 3   overview               4538 non-null   object        
 4   runtime                4538 non-null   float64       
 5   original_language      4538 non-null   object        
 6   genres                 4538 non-null   object        
 7   belongs_to_collection  4538 non-null   int64         
 8   status                 4538 non-null   object        
 9   release_date           4538 non-null   datetime64[ns]
 10  release_year           4538 non-null   int32         
 11  popularity             4538 non-null   float64       
 12  vote_count             4538 non-null   float64       
 13  vote

In [318]:
movies_top_10_percent.head()

,id,title,tagline,overview,runtime,original_language,genres,belongs_to_collection,status,release_date,...,vote_count,vote_average,production_companies,production_countries,spoken_languages,revenue,budget,return,director,casting
36871,211672,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",91.0,en,"Family, Animation, Adventure, Comedy",1,Released,2015-06-17,...,4729.0,6.4,"Universal Pictures, Illumination Entertainment",United States of America,English,1.156731e+09,74000000.0,15.631499,"[Kyle Balda, Pierre Coffin]","[Sandra Bullock, Jon Hamm, Michael Keaton, All..."
40874,297762,Wonder Woman,Power. Grace. Wisdom. Wonder.,An Amazon princess comes to the world of Man t...,141.0,en,"Action, Adventure, Fantasy",1,Released,2017-05-30,...,5025.0,7.2,"Dune Entertainment, Atlas Entertainment, Warne...",United States of America,"Deutsch, English",8.205804e+08,149000000.0,5.507251,[Patty Jenkins],"[Gal Gadot, Chris Pine, Robin Wright, Danny Hu..."
41691,321612,Beauty and the Beast,Be our guest.,A live-action adaptation of Disney's version o...,129.0,en,"Family, Fantasy, Romance",0,Released,2017-03-16,...,5530.0,6.8,"Walt Disney Pictures, Mandeville Films","United Kingdom, United States of America",English,1.262886e+09,160000000.0,7.893040,[Bill Condon],"[Emma Watson, Dan Stevens, Luke Evans, Kevin K..."
42466,339403,Baby Driver,All you need is one killer track.,After being coerced into working for a crime b...,113.0,en,"Action, Crime",0,Released,2017-06-28,...,2083.0,7.2,"Big Talk Productions, TriStar Pictures, Media ...","United Kingdom, United States of America",English,2.245113e+08,34000000.0,6.603274,[Edgar Wright],"[Ansel Elgort, Lily James, Kevin Spacey, Jamie..."
35261,177572,Big Hero 6,From the creators of Wreck-it Ralph and Frozen,The special bond that develops between plus-si...,102.0,en,"Adventure, Family, Animation, Action, Comedy",0,Released,2014-10-24,...,6289.0,7.8,"Walt Disney Pictures, Walt Disney Animation St...",United States of America,English,6.521054e+08,165000000.0,3.952154,"[Chris Williams, Don Hall]","[Scott Adsit, Ryan Potter, Daniel Henney, T.J...."


In [319]:
# Establecer la columna 'id' como índice
movies_top_10_percent.set_index('id', inplace=True)

# Resetear el índice
movies_top_10_percent.reset_index(inplace=True)

In [320]:
movies_top_10_percent.head(3)

,id,title,tagline,overview,runtime,original_language,genres,belongs_to_collection,status,release_date,...,vote_count,vote_average,production_companies,production_countries,spoken_languages,revenue,budget,return,director,casting
0,211672,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",91.0,en,"Family, Animation, Adventure, Comedy",1,Released,2015-06-17,...,4729.0,6.4,"Universal Pictures, Illumination Entertainment",United States of America,English,1.156731e+09,74000000.0,15.631499,"[Kyle Balda, Pierre Coffin]","[Sandra Bullock, Jon Hamm, Michael Keaton, All..."
1,297762,Wonder Woman,Power. Grace. Wisdom. Wonder.,An Amazon princess comes to the world of Man t...,141.0,en,"Action, Adventure, Fantasy",1,Released,2017-05-30,...,5025.0,7.2,"Dune Entertainment, Atlas Entertainment, Warne...",United States of America,"Deutsch, English",8.205804e+08,149000000.0,5.507251,[Patty Jenkins],"[Gal Gadot, Chris Pine, Robin Wright, Danny Hu..."
2,321612,Beauty and the Beast,Be our guest.,A live-action adaptation of Disney's version o...,129.0,en,"Family, Fantasy, Romance",0,Released,2017-03-16,...,5530.0,6.8,"Walt Disney Pictures, Mandeville Films","United Kingdom, United States of America",English,1.262886e+09,160000000.0,7.893040,[Bill Condon],"[Emma Watson, Dan Stevens, Luke Evans, Kevin K..."


In [321]:
# Si deseas guardar el DataFrame reducido en un nuevo archivo CSV
movies_top_10_percent.to_csv('movies_top_10_percent.csv', index=False)

In [322]:
movies_top_10_percent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4538 entries, 0 to 4537
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     4538 non-null   Int64         
 1   title                  4538 non-null   object        
 2   tagline                4538 non-null   object        
 3   overview               4538 non-null   object        
 4   runtime                4538 non-null   float64       
 5   original_language      4538 non-null   object        
 6   genres                 4538 non-null   object        
 7   belongs_to_collection  4538 non-null   int64         
 8   status                 4538 non-null   object        
 9   release_date           4538 non-null   datetime64[ns]
 10  release_year           4538 non-null   int32         
 11  popularity             4538 non-null   float64       
 12  vote_count             4538 non-null   float64       
 13  vot